<a href="https://colab.research.google.com/github/rodsei/pattern-recognition/blob/main/Reconhecimento_de_padr%C3%B5es_Ajuste_de_caracter%C3%ADsticas_(Titanic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data definition

In [4]:
import os
#IMPORT DATASET FROM KAGGLE
os.environ['KAGGLE_USERNAME'] = "XXX" # username from the json file
os.environ['KAGGLE_KEY'] = "XXX" # key from the json file
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [244]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [245]:
X.select_dtypes(include='object').columns

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')

## Atributos desejados

In [258]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
  return nome.split(',')[1].split('.')[0].strip()

def defineFaixaEtaria(idade):
  if type(idade) != float: print(idade, type(idade)); return np.nan
  if idade == np.nan: print(idade, type(idade)); return np.nan
  if idade < 12: return 'Child'
  if idade >= 12 and idade < 18: return 'Teen' 
  if idade >= 18 and idade < 45: return 'Adult'
  if idade >= 45: return 'Elder'
  
def defineFaixasPreco(preco):
  if type(preco) != float: print(preco, type(preco)); return np.nan
  if preco == np.nan: print(preco, type(preco)); return np.nan
  if preco < 8: return '3rd Class'
  if preco >= 8 and preco < 20: return '2nd Class' 
  if preco >= 20 and preco < 35: return '1st Class'
  if preco >= 35: return 'VIP'

def statusCabine(cabin):
  return 0 if type(cabin) == float else 1

def defineStatusCompanhia(tamanho):
  if type(tamanho) != int: print(tamanho, type(tamanho)); return np.nan
  if tamanho == np.nan: print(tamanho, type(tamanho)); return np.nan
  if tamanho <= 1: return 'Alone'
  if tamanho == 2: return 'Couple'
  if tamanho > 2: return 'Family'

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin', 'Age', 'Fare', 'SibSp', 'Parch']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        
        X['Has_Cabin'] = X['Cabin']
        X['Age'] = X['Age'].fillna(np.mean(X['Age']))
        X['Age_Bins'] = X['Age']
        X['Fare_Bins'] = X['Fare']
        X['FamilySize'] = X['SibSp'] + X['Parch'] + 1

        return self

    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        
        Xdrop['Age_Bins'] = Xdrop['Age_Bins'].apply(defineFaixaEtaria)
        Xdrop['Fare_Bins'] = Xdrop['Fare_Bins'].apply(defineFaixasPreco)
        Xdrop['Has_Cabin'] = Xdrop["Has_Cabin"].apply(statusCabine)
        Xdrop['Partnership'] = Xdrop["FamilySize"].apply(defineStatusCompanhia)
        return Xdrop


In [259]:
# Prepared Database
dataset = AtributosDesejados(excluirName=False).fit_transform(X)
dataset.head()

,Pclass,Name,Sex,Embarked,Has_Cabin,FamilySize,Age_Bins,Fare_Bins,Partnership
0,3,Mr,male,S,0,2,Adult,3rd Class,Couple
1,1,Mrs,female,C,1,2,Adult,VIP,Couple
2,3,Miss,female,S,0,1,Adult,3rd Class,Alone
3,1,Mrs,female,S,1,2,Adult,VIP,Couple
4,3,Mr,male,S,0,1,Adult,2nd Class,Alone


## Atributos Numéricos

In [260]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()

## Atributos Categóricos

In [261]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()

# Data preparation

## Pipeline - Trata Atributos

In [262]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])

## Pipeline - Ajuste de Características

In [264]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'classificador__n_estimators': [10, 20, 100],
    'classificador__max_depth': [3, 5, 10],
    'classificador__criterion': ['gini','entropy'],
    'classificador__n_jobs': [54]
    # 'atributosDesejados__excluirName': [False],
    # 'classificador__n_estimators': [20],
    # 'classificador__max_depth': [5], #[None] + list(range(3,5,10)),
    # 'classificador__criterion': ['gini'],
    # 'classificador__n_jobs': [-1]

}
modelo = GridSearchCV(pipetotal, param_grid=parametros, verbose=0, n_jobs=54)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

(array([0.79888268, 0.81460674, 0.8258427 , 0.80337079, 0.82022472,
        0.82681564, 0.8258427 , 0.82022472, 0.81460674, 0.83146067,
        0.81005587, 0.83146067, 0.78651685, 0.87078652, 0.79213483,
        0.8603352 , 0.81460674, 0.8258427 , 0.80898876, 0.81460674,
        0.77653631, 0.79213483, 0.84831461, 0.8258427 , 0.8258427 ,
        0.84357542, 0.79775281, 0.79775281, 0.85393258, 0.80898876,
        0.86592179, 0.79775281, 0.81460674, 0.80898876, 0.85955056,
        0.74860335, 0.78651685, 0.87078652, 0.83707865, 0.84269663,
        0.86592179, 0.78651685, 0.80337079, 0.80898876, 0.80337079,
        0.81005587, 0.85955056, 0.85955056, 0.74719101, 0.80337079]),
 0.8189655388864477,
 0.028462996345348407)

# Treinamento e Predição

In [253]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   16.2s finished
/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
